In [34]:
 
# createSqlStatements.ipynb
# erzeugt die SQL Insert Statements für die Entwicklungsumgebung
#
 
import ast
import os
import pandas as pd
from datetime import datetime
 
def readConfigFile(configFile):
 
    print("Reading congig file : %s\n" % (configFile))
    fieldExceptionDic = {}
    fileFieldExceptionDic = open(configFile, "r", encoding="utf8", errors='ignore')
    contents = fileFieldExceptionDic.read()
    fieldExceptionDic = ast.literal_eval(contents)
    fileFieldExceptionDic.close()
    return fieldExceptionDic
 
thisDate = datetime.now().strftime('%Y-%m-%d')
timeStamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
 
fileDir = "/Users/e5527h6/Documents/Programming/python/Entwicklungsdaten"
configFile = os.path.join(fileDir, "config", "dataStructure.dic")
inFileName = "Entwicklungsdaten_SQL_3.4.xlsx"
inFilePath = os.path.join(fileDir, inFileName)
try: 
    excelFile = pd.ExcelFile(inFilePath, engine='openpyxl')
except:
    print("File %s does not exist!" % (inFilePath))
    exit()
 
#  Hash-Tabelle für die HM_ABST; Keys sind die Blattnamen
sheetDic = {"INT_CTP"   : "IC",
            "XVA_QTF"   : "XVA",
            "TR"        : "mxg.TRADES",
            "TR_T-1"    : "mxg.TRADES",
            "GE"        : "zds.V_GESCH",
            "HM"        : "focus.HEDGE_MEMBERS",
            "HM_T-1"    : "focus.HEDGE_MEMBERS",
            "HR"        : "focus.HEDGE_RELATIONSHIPS",
            "HR_T-1"    : "focus.HEDGE_RELATIONSHIPS",
            "HMR"       : "focus.HEDGE_MEMBER_RESULTS",
            "HMR_T-1"   : "focus.HEDGE_MEMBER_RESULTS",
            "HETS"      : "focus.HEDGE_EFFECTIVENESS_TEST_SETTINGS",
            "HETS_T-1"  : "focus.HEDGE_EFFECTIVENESS_TEST_SETTINGS",
            "AHMR"      : "focus.ANALYSIS_A_HEDGE_MEMBER_RESULTS",
            "RARA"      : "focus.REGRESSION_ANALYSIS_RESULTS_ARCHIVE"
           }
 
# Datei mit Felddefinitionen einlesen
fieldExceptionDic = readConfigFile(configFile)
 
cellCounterFile = 0
for sheet in excelFile.sheet_names:
    try:
        df = pd.read_excel(excelFile, sheet, header=None)
    except Exception:
        print("Fields sheet %s does not exist in file %s" % (sheet, inFilePath))
        continue
    if sheet in sheetDic.keys():
        sheetName = sheetDic[sheet]
        print("Processing sheet %s." % sheet)
    else:
        print("Sheet %s will not be processed!" % sheet)
        continue
    # get PER_DATE_POSITION
    perDatePosition = str(df.iat[2, 0])[:10]
    # output file
    if not os.path.exists(os.path.join(fileDir, 'SQL_Upload')):
        os.makedirs(os.path.join(fileDir, 'SQL_Upload'))
    outFilePath = os.path.join(fileDir, 'SQL_Upload', 'SQL_Upload_' + sheetName + '#' + perDatePosition + '_cr_' + thisDate + '.sql')
    outFileHandle = open(outFilePath,'w')
    cellCounterSheet = 0
    headerList = []
    headerStr = ''
    viewNameStage = ''
    viewNameQuery = ''
    for index, row in df.iterrows():
        # extract table name
        if index == 0:
            viewNameStage = df.iat[index, 0]
            viewNameQuery = viewNameStage[:-4]
            perDatePosition = df.iat[index + 2, 0].date()
        # extract header fields
        if index == 1:
            headerIdx = 0
            for headerField in row:
                headerList.append(str(headerField))
                headerIdx += 1
            headerStr = "(" + ",".join(headerList) + ")"
            print("---" + viewNameStage + "---")
        valueList = []
        valueStr = ''
        if index > 1:
            valueIdx = 0
            for valueField in row:
                # NULL-Werte durch NULL ersetzen, sind nicht SQL kompatibel
                if pd.notna(valueField):
                    valueField = valueField
                else:
                    valueField = 'NULL'
                # MS Access NULL in SQL NULL umwandeln
                if valueField == '<null>' or valueField == 'NULL':
                    valueList.append(str('NULL'))
                # leere Datumsfelder mit NULL füllen
                elif ( headerList[valueIdx] in fieldExceptionDic[viewNameStage] and
                     (fieldExceptionDic[viewNameStage][headerList[valueIdx]] == "datetime" or
                     fieldExceptionDic[viewNameStage][headerList[valueIdx]] == "date" ) and
                     valueField == '' ):
                    valueList.append(str('NULL'))
                # Datumsfelder kürzen aus 23 Stellen  2022-06-14 10:16:12.570
                elif ( headerList[valueIdx] in fieldExceptionDic[viewNameStage] and
                     fieldExceptionDic[viewNameStage][headerList[valueIdx]] == "datetime" and
                     valueField != '' ):
                    valueList.append("'" + str(valueField)[0:23] + "'")
                # String Felder mit Hochkomma importieren
                elif headerList[valueIdx] in fieldExceptionDic[viewNameStage]:
                    valueList.append("'" + str(valueField) + "'")
                # leere numerische Felder mit 0 füllen
                elif valueField == '':
                    valueList.append(str(0))
                # den Rest einfach importieren, wie er ist
                else:
                    valueList.append(str(valueField))
                valueIdx += 1
            valueStr = "(" + ",".join(valueList) + ")"
        if index == 1:
            outFileHandle.write("begin transaction\ngo\n")
            outFileHandle.write("use RiCo_PoDIuM\n")
            outFileHandle.write("SELECT *  FROM " + viewNameStage + " WHERE PER_DATE_POSITION = '" + str(perDatePosition) + "';\n")
            outFileHandle.write("SELECT *  FROM " + viewNameQuery + " WHERE PER_DATE_POSITION = '" + str(perDatePosition) + "';\n")
            outFileHandle.write("insert into base.V_LOG_DLV ( PER_DATE_POSITION, STATUS_FLAG, SOURCE_OBJ, TARGET_OBJ, JOB_NAME, ERROR_DESC, MISC_DESC, TIME_STAMP)\n")
            outFileHandle.write("values\n")
            outFileHandle.write("( '" + str(perDatePosition) + "', 'S', 'Manual Insert', '" + viewNameStage + "', 'Manual', '', '', GETDATE())\n\n")
        if index > 1 and pd.notna(df.iat[index, 2]):
            outFileHandle.write("insert into " + viewNameStage + " " + headerStr + " values " + valueStr + "\n")
            cellCounterSheet += 1
            cellCounterFile += 1
    outFileHandle.write("\ninsert into base.V_LOG_DLV ( PER_DATE_POSITION, STATUS_FLAG, SOURCE_OBJ, TARGET_OBJ, JOB_NAME, ERROR_DESC, MISC_DESC, TIME_STAMP )\n")
    outFileHandle.write("values\n")
    outFileHandle.write("( '" + str(perDatePosition) + "', 'E', 'Manual Insert', '" + viewNameStage + "', 'Manual', '', '', GETDATE())\n\n")
    outFileHandle.write("SELECT *  FROM " + viewNameStage + " WHERE PER_DATE_POSITION = '" + str(perDatePosition) + "';\n")
    outFileHandle.write("SELECT *  FROM " + viewNameQuery + " WHERE PER_DATE_POSITION = '" + str(perDatePosition) + "';\n\n")
    outFileHandle.write("commit\ngo\n")
    outFileHandle.close()


Reading congig file : /Users/e5527h6/Documents/Programming/python/Entwicklungsdaten/config/dataStructure.dic

Sheet README will not be processed!
Sheet CreateTestData will not be processed!
Processing sheet XVA_QTF.
---dbo.XVA_QANTIFI_ACT---
Processing sheet TR.
---mxg.V_TRADES_DLV---
Processing sheet TR_T-1.
---mxg.V_TRADES_DLV---
Processing sheet GE.
---zds.V_GESCH_DLV---
Processing sheet HM.
---focus.V_HEDGE_MEMBERS_DLV---
Processing sheet HM_T-1.
---focus.V_HEDGE_MEMBERS_DLV---
Processing sheet HR.
---focus.V_HEDGE_RELATIONSHIPS_DLV---
Processing sheet HR_T-1.
---focus.V_HEDGE_RELATIONSHIPS_DLV---
Processing sheet HMR.
---focus.V_HEDGE_MEMBER_RESULTS_DLV---
Processing sheet HMR_T-1.
---focus.V_HEDGE_MEMBER_RESULTS_DLV---
Processing sheet HETS.
---focus.V_HEDGE_EFFECTIVENESS_TEST_SETTINGS_DLV---
Processing sheet HETS_T-1.
---focus.V_HEDGE_EFFECTIVENESS_TEST_SETTINGS_DLV---
Processing sheet AHMR.
---focus.V_ANALYSIS_A_HEDGE_MEMBER_RESULTS_DLV---
Processing sheet RARA.
---focus.V_REGR